In [1]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.nn as nn
import torch 
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms


In [2]:
!ls

batch_test.ipynb   data       training_data2.pt  training_data.pt
catsvcsdogs.ipynb  PetImages  training_data.npy


In [3]:
training_data = torch.load('training_data2.pt')

In [4]:
training_data[0][0].shape

torch.Size([50, 50])

In [5]:
training_data[0][0].view(1,50,50).shape

torch.Size([1, 50, 50])

In [6]:
print(len(training_data))

24946


In [7]:
for x in range(len(training_data)): 
    training_data[x][0] = training_data[x][0].view(1,50,50)
    

In [8]:
training_data[0][0].shape

torch.Size([1, 50, 50])

In [48]:
#divisão dos dados de teste e dados de treino
#traning data and validation data
validatation_percent = 0.2
validatation_size    = int(len(X)*validatation_percent)
print(validatation_size)

4989


In [99]:
train_dataset = training_data[:-validatation_size]

test_dataset = training_data[-validatation_size:]




In [104]:
print('tamanho dataset de treino: ', len(train_dataset))
print('tamanho dataset test: ', len(test_dataset))



tamanho dataset de treino:  19957
tamanho dataset test:  4989


In [105]:
data_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=100
)

In [109]:
len(data_loader.dataset)


19957

In [110]:
batch = next(iter(data_loader))

In [111]:
images, labels = batch

In [112]:
images.shape

torch.Size([100, 1, 50, 50])

In [113]:
labels.shape

torch.Size([100])

In [58]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()

        # self.image_flattened = self.num_flat_features(_shape_image)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3)  
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3)

        self.fc1   = nn.Linear(12*11*11, 128)
        self.fc2   = nn.Linear(128, 64)
        self.output   = nn.Linear(64, 2)

    def forward(self, t):
        t = F.relu(self.conv1(t.float()))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = F.relu(self.fc1(t.reshape(-1, 12*11*11)))
        t = F.relu(self.fc2(t))
        t = self.output(t)

        return F.softmax(t, dim=1)

    def num_flat_features(self, _shape_image):
        size = _shape_image[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [89]:
network = SimpleCNN()

In [90]:
pred = network(images)
pred

tensor([[9.9947e-01, 5.2811e-04],
        [9.9226e-01, 7.7369e-03],
        [9.9058e-01, 9.4228e-03],
        [9.9736e-01, 2.6443e-03],
        [9.3604e-01, 6.3961e-02],
        [9.9845e-01, 1.5549e-03],
        [9.9995e-01, 4.6707e-05],
        [9.9909e-01, 9.0666e-04],
        [9.9902e-01, 9.8037e-04],
        [9.9737e-01, 2.6268e-03]], grad_fn=<SoftmaxBackward>)

In [91]:
pred.argmax(dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [92]:
labels

tensor([1, 0, 1, 1, 0, 0, 1, 1, 0, 0])

In [93]:
pred.argmax(dim=1).eq(labels).sum()

tensor(5)

In [94]:
loss = F.cross_entropy(pred, labels)
loss.item()

0.8155854940414429

In [95]:
#entendendo o calculo dos gradientes
print(network.conv1.weight.grad)

None


In [96]:
loss.backward() # calculo dos gradientes

In [97]:
print(network.conv1.weight.grad)# representando o kerneis que eu setei na primeira camada de convolução

tensor([[[[ 7.7288e-03,  9.4204e-03,  7.2584e-03],
          [ 3.7312e-03,  6.7548e-03,  7.1692e-03],
          [ 4.4665e-03,  5.7432e-03,  8.3368e-03]]],


        [[[-9.5167e-03, -8.8802e-03, -5.3966e-03],
          [-3.9754e-03, -6.6999e-03, -6.9883e-03],
          [-1.1671e-03, -4.3982e-03, -4.3649e-03]]],


        [[[ 3.2656e-03,  1.1537e-03,  4.6737e-05],
          [ 3.7965e-03,  1.7100e-03, -1.4718e-03],
          [ 1.9210e-03, -9.7714e-04, -2.9008e-04]]],


        [[[ 1.8556e-02,  2.0214e-02,  1.5636e-02],
          [ 1.2630e-02,  1.3462e-02,  1.1815e-02],
          [ 9.4861e-03,  1.3071e-02,  1.2236e-02]]],


        [[[-1.7835e-02, -1.2983e-02, -1.5314e-02],
          [-1.4178e-02, -1.0312e-02, -1.2344e-02],
          [-1.1311e-02, -1.1521e-02, -1.4355e-02]]],


        [[[ 4.8542e-03,  6.4347e-03,  8.6560e-03],
          [ 4.1868e-03,  3.1218e-03,  4.3148e-03],
          [-1.4028e-04,  3.8202e-03,  6.3113e-03]]]])


In [98]:
print(network.conv1.weight.grad.shape) # 

torch.Size([6, 1, 3, 3])


In [70]:
#Para atulizar os pesos da rede usa-se um otimizador
optimizer = torch.optim.Adam(network.parameters(), lr=0.01)

In [71]:
loss.item()

0.8132615089416504

In [72]:
pred.argmax(dim=1).eq(labels).sum()

tensor(5)

In [73]:
optimizer.step() #atualiza os pesos na rede

In [74]:
pred = network(images)
loss = F.cross_entropy(pred, labels)

In [75]:
loss.item() #O valor abaixou

0.7155604958534241

In [76]:
pred.argmax(dim=1).eq(labels).sum()

tensor(6)

In [114]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [123]:
#treinando todos batches em uma unica epoch

net = SimpleCNN()

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100)
optim = torch.optim.Adam(network.parameters(), lr=0.01)

for epoch in range(3):
    for batch in train_loader:
        
        images, labels = batch
        preds = net(images)
        loss_func = F.cross_entropy(preds, labels)

        optim.zero_grad()
        loss_func.backward() # Calcula os gradientes
        optim.step() #atualiza os pesos
        
    

0
0.7180442214012146
51
0
0.724414050579071
52
0
0.7512136697769165
50
0
0.761513888835907
48
0
0.7006736993789673
54
0
0.7685655951499939
46
0
0.68153977394104
60
0
0.7975253462791443
42
0
0.712160050868988
56
0
0.7327563762664795
50
0
0.7492381930351257
46
0
0.7823872566223145
45
0
0.6952136754989624
56
0
0.7255175113677979
53
0
0.6976770162582397
56
0
0.6855154633522034
61
0
0.7218544483184814
53
0
0.6927326321601868
59
0
0.7226680517196655
52
0
0.6932046413421631
58
0
0.7177224159240723
52
0
0.7256126999855042
53
0
0.8039790987968445
44
0
0.8541256189346313
36
0
0.7201734185218811
53
0
0.781475305557251
45
0
0.74747633934021
50
0
0.7944180965423584
41
0
0.7442733645439148
51
0
0.6712265014648438
58
0
0.7946435809135437
44
0
0.7343109846115112
51
0
0.7911770343780518
43
0
0.779391884803772
45
0
0.7649511694908142
50
0
0.764169454574585
47
0
0.7292197346687317
54
0
0.827568531036377
36
0
0.737443745136261
52
0
0.727118730545044
53
0
0.7081989049911499
54
0
0.7979742288589478
45
0
0.7

1
0.6908101439476013
58
1
0.7071584463119507
55
1
0.7155356407165527
53
1
0.7706034183502197
48
1
0.7675769329071045
47
1
0.7123197317123413
55
1
0.7222959995269775
52
1
0.7650026679039001
47
1
0.7295730113983154
53
1
0.7528713941574097
47
1
0.7010387182235718
58
1
0.7199646234512329
53
1
0.7326568365097046
51
1
0.6841889023780823
60
1
0.7837421298027039
47
1
0.7460387349128723
47
1
0.732086718082428
52
1
0.7095041871070862
55
1
0.746253490447998
49
1
0.7715147137641907
47
1
0.7104511857032776
55
1
0.7698498368263245
45
1
0.7778505086898804
46
1
0.7153061628341675
53
1
0.7779698967933655
49
1
0.7268884778022766
53
1
0.7423632144927979
50
1
0.7172830104827881
53
1
0.7720449566841125
45
1
0.6848912239074707
57
1
0.738401472568512
50
1
0.7351930141448975
52
1
0.750988245010376
53
1
0.7625259160995483
45
1
0.8333894610404968
40
1
0.7311527729034424
55
1
0.7866888642311096
44
1
0.7438690066337585
53
1
0.6810553073883057
61
1
0.7145032286643982
53
1
0.7404125928878784
52
1
0.7878623008728027